In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import itertools
import random

import sys
sys.path.append('/Users/andrew/Desktop/sudoku/src/sudoku')

from board import Board
from solutions import Solutions
import utils

In [2]:
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)

In [120]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [4]:
def vector_encode(board_string):
    dim_x, dim_y, board = board_string.split('.')
    max_digit = int(dim_x) * int(dim_y)
    vector = np.zeros((max_digit*len(board),), dtype=np.float64)
    for i in range(len(board)):
        if board[i] != '0':
            vector[i*max_digit + int(board[i]) - 1] = 1
        else:
            vector[i*max_digit:(i+1)*max_digit] = 1/max_digit
    return vector

def get_board_entries(board_string):
    return np.array(list(board_string[4:]), dtype=np.int)

In [5]:
def shuffle_numbers(seed_board_string, number_orders):
    """
    Given a seed_board string and a string with the new order for top row of board, generates derivative board
    e.g.
    >>> shuffle_numbers('2.2.0204030120400400', ['2143'])
    ['2.2.0103040210300300']
    """
    max_digit = int(seed_board_string[0]) * int(seed_board_string[2])
    digits = seed_board_string[4:]
    assert np.all([int(digits[i]) == i+1 or int(digits[i]) == 0 for i in range(max_digit)]) # check if seed
    if type(number_orders) == str:
        number_orders = [number_orders]
        
    all_digits = {str(i) for i in range(1, max_digit+1)}
    shuffled_board_strings = []
    for number_order in number_orders:
        assert len(number_order) == max_digit
        mapping = {str(i+1): str(number_order[i]) for i in range(max_digit)}
        assert set(mapping.values()) == all_digits
        shuffled_board_string = seed_board_string[:4]
        shuffled_board_string += ''.join([mapping[d] if d != '0' else '0' for d in digits])
        shuffled_board_strings.append(shuffled_board_string)
    
    return shuffled_board_strings

In [6]:
def generate_derivatives(puzzles, solutions, num_derivatives=0):
    """
    puzzles: list of seed Boards
    solutions: Solutions
    num_derivatives: max number of derivatives to generate per seed. Default: 0 -> all
    returns
    derivative_puzzles: dict - seed board string -> list of derivative board strings
    derivative_puzzle_solutions: dict - derivative board string -> derivative board solution string
    """
    max_digit = puzzles[0].board.shape[0]
    all_digits_string = ''.join([str(i) for i in range(1, max_digit+1)])
    permutations = [''.join(lst) for lst in itertools.permutations(all_digits_string, max_digit)]
    random.shuffle(permutations)
    if num_derivatives > 0:
        permutations = permutations[:num_derivatives]
    derivative_puzzles = {}
    derivative_puzzle_solutions = {}
    for puzzle in puzzles:
        puzzle_string = puzzle.stringify()
        derivatives = shuffle_numbers(puzzle_string, permutations)
        derivative_solutions = shuffle_numbers(solutions[puzzle].stringify(), permutations)
        derivative_puzzles[puzzle_string] = derivatives
        for i in range(len(permutations)):
            derivative_puzzle_solutions[derivatives[i]] = derivative_solutions[i]
    return derivative_puzzles, derivative_puzzle_solutions

In [7]:
def split_data(data, boundaries):
    """
    Shuffles and splits data according to the sorted boundaries set
    """
    assert boundaries[0] > 0 and boundaries[-1] < 1 and boundaries == sorted(boundaries)
    assert len(data) > len(boundaries)
    
    data = list(data)
    np.random.shuffle(data)
    
    split = []
    last_boundary = 0
    for boundary in boundaries + [1]:
        next_boundary = int(len(data) * boundary)
        split.append(data[last_boundary:next_boundary])
        last_boundary = next_boundary
    return split

In [8]:
def generate_dataset(seed_puzzles, solutions, boundaries, num_derivatives=0):
    """
    puzzles: list of puzzle
    """
    derivative_puzzles, derivative_puzzle_solutions = generate_derivatives(seed_puzzles, solutions, num_derivatives)

    train_seeds, valid_seeds, test_seeds = split_data([p.stringify() for p in seed_puzzles], boundaries)
    train_seeds_derivs = utils.flatten([derivative_puzzles[puzzle] for puzzle in train_seeds])
    train_puzzles, valid_deriv_puzzles, test_deriv_puzzles = split_data(train_seeds_derivs, boundaries)

    train = {puzzle: derivative_puzzle_solutions[puzzle] for puzzle in train_puzzles}
    valid_deriv = {puzzle: derivative_puzzle_solutions[puzzle] for puzzle in valid_deriv_puzzles}
    test_deriv = {puzzle: derivative_puzzle_solutions[puzzle] for puzzle in test_deriv_puzzles}

    valid_nonderiv_puzzles = utils.flatten([derivative_puzzles[puzzle] for puzzle in valid_seeds])
    valid_nonderiv = {puzzle: derivative_puzzle_solutions[puzzle] for puzzle in valid_nonderiv_puzzles}
    test_nonderiv_puzzles = utils.flatten([derivative_puzzles[puzzle] for puzzle in test_seeds])
    test_nonderiv = {puzzle: derivative_puzzle_solutions[puzzle] for puzzle in test_nonderiv_puzzles}

    # sanity check
    assert not train.keys() & valid_deriv.keys()
    assert not train.keys() & test_deriv.keys()
    assert not train.keys() & valid_nonderiv.keys()
    assert not train.keys() & test_nonderiv.keys()
    assert not valid_deriv.keys() & test_deriv.keys()
    assert not valid_deriv.keys() & valid_nonderiv.keys()
    assert not valid_deriv.keys() & test_nonderiv.keys()
    assert not test_deriv.keys() & valid_nonderiv.keys()
    assert not test_deriv.keys() & test_nonderiv.keys()
    assert not valid_nonderiv.keys() & test_nonderiv.keys()
    
    return train, valid_deriv, test_deriv, valid_nonderiv, test_nonderiv

In [10]:
solutions = Solutions('/Users/andrew/Desktop/sudoku/data/solutions5.txt')
puzzles = solutions.get_puzzles_by_hints()
for hints in sorted(puzzles):
    print(hints, len(puzzles[hints]))

train, valid_deriv, test_deriv, valid_nonderiv, test_nonderiv = generate_dataset(puzzles[4], solutions, [.7, .8])

4 64
5 357
6 883
7 1584
8 2384
9 3309
10 4149
11 4754
12 4841
13 3741
14 1391
15 192
16 12


In [121]:
class Linear(nn.Module):
    def __init__(self, max_digit):
        super(Linear, self).__init__()
        self.max_digit = max_digit
        self.linear = nn.Linear(self.max_digit**3, self.max_digit**3)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, X):
        pre_output = self.linear(X).reshape(X.shape[0], self.max_digit**2, self.max_digit)
        return self.softmax(pre_output)
#         predictions = [softmax(pre_output[i*self.max_digit:(i+1)*self.max_digit]) for i in range(self.max_digit**2)]
        
#         return torch.stack(predictions)

In [117]:
a = torch.randn(2, 3, 4)
b = nn.Softmax(dim=2)(a)

In [118]:
a

tensor([[[ 0.7624,  0.9520,  0.8139,  0.5260],
         [-0.5523, -0.6327,  0.0902,  0.5165],
         [-0.8378,  0.2764, -1.8262, -0.6584]],

        [[-0.1834, -0.3537,  1.6652,  0.3598],
         [ 0.8794, -1.0414,  0.7348,  2.2601],
         [-1.0239, -0.6942, -0.8388, -1.1062]]])

In [119]:
b

tensor([[[0.2468, 0.2984, 0.2599, 0.1949],
         [0.1485, 0.1370, 0.2823, 0.4323],
         [0.1781, 0.5426, 0.0663, 0.2131]],

        [[0.1009, 0.0851, 0.6405, 0.1736],
         [0.1670, 0.0245, 0.1445, 0.6641],
         [0.2215, 0.3080, 0.2665, 0.2040]]])

In [95]:
def generate_XY(puzzles):
    """
    puzzles: dict - puzzle board string -> solution board string
    """
    keys = sorted(puzzles)
    X = torch.tensor([vector_encode(p) for p in keys])
    Y = torch.tensor([get_board_entries(puzzles[p]) for p in keys], dtype=torch.int64) - 1
    return X, Y

In [146]:
max_digit = 4
model = Linear(max_digit)
model.double()

optimizer = optim.LBFGS(model.parameters(), lr=0.8)
X, Y = generate_XY(train)
# X = X[:2]
# Y = Y[:2]

def closure():
    optimizer.zero_grad()
    prediction = model(X)
    total_cells = Y.shape[0]*Y.shape[1]
    flattened_prediction = prediction.reshape(total_cells, max_digit)
    flattened_Y = Y.reshape(total_cells)
    loss = nn.functional.nll_loss(flattened_prediction, flattened_Y)
    print(loss)
    loss.backward()
    return loss

In [ ]:
for i in range(50):
    optimizer.step(closure)

tensor(-0.2502, grad_fn=<NllLossBackward>)
tensor(-0.2503, grad_fn=<NllLossBackward>)
tensor(-0.2504, grad_fn=<NllLossBackward>)
tensor(-0.2506, grad_fn=<NllLossBackward>)
tensor(-0.2507, grad_fn=<NllLossBackward>)
tensor(-0.2508, grad_fn=<NllLossBackward>)
tensor(-0.2509, grad_fn=<NllLossBackward>)
tensor(-0.2510, grad_fn=<NllLossBackward>)
tensor(-0.2511, grad_fn=<NllLossBackward>)
tensor(-0.2512, grad_fn=<NllLossBackward>)
tensor(-0.2513, grad_fn=<NllLossBackward>)
tensor(-0.2514, grad_fn=<NllLossBackward>)
tensor(-0.2515, grad_fn=<NllLossBackward>)
tensor(-0.2516, grad_fn=<NllLossBackward>)
tensor(-0.2518, grad_fn=<NllLossBackward>)
tensor(-0.2519, grad_fn=<NllLossBackward>)
tensor(-0.2520, grad_fn=<NllLossBackward>)
tensor(-0.2521, grad_fn=<NllLossBackward>)
tensor(-0.2522, grad_fn=<NllLossBackward>)
tensor(-0.2523, grad_fn=<NllLossBackward>)
tensor(-0.2524, grad_fn=<NllLossBackward>)
tensor(-0.2525, grad_fn=<NllLossBackward>)
tensor(-0.2526, grad_fn=<NllLossBackward>)
tensor(-0.2

In [131]:
model(X[:1])

tensor([[[4.8582e-24, 3.9519e-22, 4.2952e-24, 1.0000e+00],
         [8.2320e-14, 1.0000e+00, 3.0586e-08, 2.4407e-12],
         [1.0000e+00, 4.2197e-22, 1.7555e-21, 1.6835e-21],
         [1.8346e-11, 1.3179e-06, 1.0000e+00, 6.1353e-12],
         [4.1912e-12, 1.6680e-06, 1.0000e+00, 1.1903e-12],
         [1.0000e+00, 6.1607e-24, 9.3786e-22, 1.7533e-22],
         [5.0747e-23, 2.7342e-23, 4.3179e-23, 1.0000e+00],
         [1.0154e-13, 1.0000e+00, 2.5331e-08, 1.1036e-12],
         [1.0000e+00, 1.0886e-23, 6.4354e-23, 1.0542e-22],
         [1.3856e-12, 1.4528e-06, 1.0000e+00, 1.4370e-12],
         [6.6631e-12, 1.0000e+00, 1.3912e-06, 1.0697e-11],
         [2.2263e-22, 2.3738e-23, 1.0526e-22, 1.0000e+00],
         [7.7964e-13, 1.0000e+00, 3.1206e-08, 1.2690e-12],
         [5.9563e-22, 9.2355e-22, 2.1416e-22, 1.0000e+00],
         [2.1281e-11, 2.1017e-06, 1.0000e+00, 1.8384e-12],
         [1.0000e+00, 6.7983e-22, 6.6278e-23, 4.6920e-22]]],
       grad_fn=<SoftmaxBackward>)

In [140]:
np.argmax(model(X[2:3]).detach().numpy(), axis=2) + 1

RuntimeError: cannot unsqueeze empty tensor

In [141]:
print(Y[2] + 1)

IndexError: index 2 is out of bounds for dimension 0 with size 2

In [45]:
max_digit = 4
model = Linear(max_digit)
model.double()

optimizer = optim.LBFGS(model.parameters(), lr=0.8)
X = torch.tensor(vector_encode('2.2.0134432100400010'))
Y = torch.tensor(get_board_entries('2.2.2134432112433412'), dtype=torch.int64) - 1

print(X.shape)
print(Y.shape)

def closure():
    optimizer.zero_grad()
    prediction = model(X)
    print(prediction.shape)
    loss = nn.functional.nll_loss(prediction, Y)
#     print(loss)
    loss.backward()
    return loss

for i in range(4):
    optimizer.step(closure)

torch.Size([64])
torch.Size([16])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])


/Users/andrew/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
for i in range(15):
    print('STEP: ', i)
    def closure():
        optimizer.zero_grad()
        out = seq(input)
        loss = criterion(out, target)
        print('loss:', loss.item())
        loss.backward()
        return loss
    optimizer.step(closure)
    # begin to predict, no need to track gradient here
    with torch.no_grad():
        future = 1000
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print('test loss:', loss.item())
        y = pred.detach().numpy()
    # draw the result
    plt.figure(figsize=(30,10))
    plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
    plt.xlabel('x', fontsize=20)
    plt.ylabel('y', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    def draw(yi, color):
        plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
        plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
    draw(y[0], 'r')
    draw(y[1], 'g')
    draw(y[2], 'b')
    plt.savefig('predict%d.pdf'%i)
    plt.close()

In [61]:
board = Board.loadFromString('2.2.0134432100400010')


In [62]:
board

array([[0, 1, 3, 4],
       [4, 3, 2, 1],
       [0, 0, 4, 0],
       [0, 0, 1, 0]], dtype=int8)

In [63]:
'2.2.2134432112433412'

'2.2.2134432112433412'

In [112]:
def shuffle_numbers(seed_board_string, number_orders):
    max_digit = int(seed_board_string[0]) * int(seed_board_string[2])
    digits = seed_board_string[4:]
    assert np.all([int(digits[i]) == i+1 or int(digits[i]) == 0 for i in range(max_digit)]) # check if seed
    if type(number_orders) == str:
        number_orders = [number_orders]
        
    all_digits = {str(i) for i in range(1, max_digit+1)}
    shuffled_board_strings = []
    for number_order in number_orders:
        assert len(number_order) == max_digit
        mapping = {str(i+1): str(number_order[i]) for i in range(max_digit)}
        assert set(mapping.values()) == all_digits
        shuffled_board_string = seed_board_string[:4]
        shuffled_board_string += ''.join([mapping[d] for d in digits])
        shuffled_board_strings.append(shuffled_board_string)
    
    return shuffled_board_strings

In [53]:
list(range(5))[5:6]

[]